In [ ]:
!pip install langid
!pip install --upgrade nltk

In [8]:
# Importing libraries
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from textblob import Word, TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings('ignore')
import re
import zipfile
import concurrent.futures
import langid

In [9]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('vader_lexicon', quiet=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [4]:
# read data
data = pd.read_csv('combined_dataset.csv')

# show sample
data.head()

,site,role,company_name,location,job_description,salary_range,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
0,glassdoor,Senior Advisor Crime Data & Analysis,Metropolitan Police Department3.4 ★,"Washington, DC",There is only one city in the country that can...,$127K - $178K (Employer est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,glassdoor,Data Analyst – Data Entry,Juristat,Remote,Juristat is building an innovative suite of pr...,$30K - $35K (Employer est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,glassdoor,Business Systems Analyst,Distributed Solar Development4.1 ★,"New York, NY",Join AZAD Technology Partners as a Business Sy...,$130K - $160K (Employer est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,glassdoor,Data Analyst,Haberfeld4.5 ★,"Lincoln, NE","Who We Are:\nAt Haberfeld we align marketing, ...",$54K - $82K (Glassdoor est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glassdoor,Senior Advisor Crime Data & Analysis,LeewayHertz4.1 ★,"San Francisco, CA",There is only one city in the country that can...,$58K - $88K (Glassdoor est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def text_preprocessing(text, result = 'clean_tokens'):
    """
    Preprocesses the raw text applying the following steps: convert to string, remove newline characters, standardize, remove numbers, punctuation and stopwords, stem and lemmatize the words.

    Parameters:
    - text (str): The raw text to be preprocessed.
    - result (str): The step to include in the output. Possible values: 'text', 'combined_text', 'standardized_text', 'no_punctuation',
                                                                        'no_numbers', 'no_stopwords', 'stemmed_tokens', 'lemmatized_tokens', 'clean_tokens'.
                    Default is 'clean_tokens'.

    Returns:
    - str or list: The result of the specified step applied.
    """
    text = str(text)
    if result == 'text':
      return text
    # remove newline characters
    combined_text = text.replace('\n', ' ')
    if result == 'combined_text':
      return combined_text

    # standardization of letters (make lowercase)
    standardized_text = combined_text.lower()
    if result == 'standardized_text':
      return standardized_text

    # remove punctuation
    no_punctuation = re.sub(r'[^\w\s]', '', standardized_text)
    if result == 'no_punctuation':
      return no_punctuation

    # remove numbers
    no_numbers = re.sub(r'\d', '', no_punctuation)
    if result == 'no_numbers':
      return no_numbers

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    no_stopwords = " ".join([word for word in no_numbers.split() if word not in stop_words])
    if result == 'no_stopwords':
      return no_stopwords

    # spacy tokenization
    nlp = spacy.load('en_core_web_sm')
    doc_tokenize = nlp(no_stopwords)
    tokens = [token.text for token in doc_tokenize]
    if result == 'tokens':
      return tokens

    # stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    if result == 'stemmed_tokens':
      return stemmed_tokens

    # lemmatization
    sentence = " ".join(tokens)
    doc_lemmitize = nlp(sentence)
    lemmatized_tokens = [token.lemma_ for token in doc_lemmitize]
    if result == 'lemmatized_tokens':
      return lemmatized_tokens

    # final cleaning: remove empty strings, single letters and duplicates
    clean_tokens = [token for token in list(set(lemmatized_tokens)) if token.strip() != '' and len(token) > 1]
    if result == 'clean_tokens':
      return clean_tokens

def is_eng(text):
    """
    Checks if the input text is in the English language.

    Parameters:
    - text (str): The input text to be checked for English language.

    Returns:
    - bool: True if the text is identified as English, False otherwise.
    """
    text = str(text)
    # Detects the language of the text
    lang, confidence = langid.classify(str(text))

    # Check if the language is English
    return lang == 'en'

def parallel_processing(iterable, processing_function, *processing_args, workers=3):
    """
    Process elements in parallel using ThreadPoolExecutor.

    Parameters:
    - iterable (iterable): The iterable containing elements to be processed.
    - processing_function (function): The processing function to apply to the elements.
    - processing_args: Additional arguments to pass to the processing function.
    - workers (int, optional): The number of parallel workers. Default is 3.

    Returns:
    - list: A list of processed elements after applying the specified processing function in parallel.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        processed_elements = list(executor.map(lambda element: processing_function(element, *processing_args), iterable))

    return processed_elements

exchange_rates = {
    'United Kingdom': {'currency_name': 'Pounds Sterling GBP', 'currency_symbol': '£', 'exchange_rate': 0.79},
    'United States': {'currency_name': 'US Dollar USD', 'currency_symbol': '$', 'exchange_rate': 1.00},
    'Eurozone': {'currency_name': 'Euro EUR', 'currency_symbol': '€', 'exchange_rate': 0.92},
    'India': {'currency_name': 'Indian Rupee INR', 'currency_symbol': '₹', 'exchange_rate': 83.30}
}

def is_valid_currency_symbol(symbol, exchange_rates = exchange_rates):
    """
    Check if a given currency symbol is present in the exchange rates dictionary.

    Parameters:
    - symbol (str): The currency symbol to check.
    - exchange_rates (dict): Dictionary containing currency information.

    Returns:
    - bool: True if the symbol is present, False otherwise.
    """
    for currency_info in exchange_rates.values():
        if 'currency_symbol' in currency_info and currency_info['currency_symbol'] == symbol:
            return True
    return False

import re

def extract_salary(text, exchange_rates = exchange_rates):
    """
    Extracts salary information from a given text.

    Parameters:
    - text (str): The input text containing salary information.
    - exchange_rates (dict): Dictionary containing currency information.

    Returns:
    - tuple or None: A tuple containing (min_salary_usd, max_salary_usd) with the extracted
      minimum and maximum salary values in USD. Returns (None, None) if no valid
      salary information is found.
    """
    text = str(text)
    # Define the pattern for salary information
    get_salary_pattern = re.compile(r'([\$€£₹])\s*([\d,]+[a-zA-Z]?)\s*-\s*([\$€£₹])?\s*([\d,]+[a-zA-Z]?)?')

    # Define the pattern for extracting numeric values with optional suffix
    get_number_pattern = re.compile(r'\b(\d+(?:,\d{3})*(?:\.\d+)?)(?:[a-zA-Z]+)?\b')

    # Define the short scale mapping for suffixes
    short_scale_mapping = {'K': 1e3, 'M': 1e6, 'B': 1e9, 'T': 1e12, 'L': 1e5}

    # Search for the salary pattern in the text
    match = get_salary_pattern.search(text)

    if match:
        # Extract relevant information from the matched groups
        min_currency_symbol = match.group(1)
        min_salary_str = match.group(1) + match.group(2)
        max_salary_str = match.group(3) + match.group(4) if match.group(3) else min_salary_str

        try:
            # Extract numeric values with optional suffix
            min_salary = float(get_number_pattern.search(min_salary_str).group(1).replace(',', ''))
            max_salary = float(get_number_pattern.search(max_salary_str).group(1).replace(',', ''))

            # Check if there's a suffix and apply the multiplier if needed
            suffix = min_salary_str[-1]
            if suffix.upper() in short_scale_mapping:
                min_salary *= short_scale_mapping[suffix.upper()]
                max_salary *= short_scale_mapping[suffix.upper()]
            elif not suffix.isdigit() and len(suffix) >= 1:
                print('Unknown suffix in the salary range: ' + str(suffix) + ' Update short_scale_mapping dictionary.')

        except (ValueError, AttributeError):
            return (None, None)

        # Check if the currency symbol is valid
        if not is_valid_currency_symbol(min_currency_symbol):
            print('Unknown currency symbol: ' + str(min_currency_symbol) + ' Update exchange_rates dictionary and get_salary_pattern.')
            print('Available currency symbols:', list(exchange_rates.keys()))
            return (None, None)

        # Find currency information based on the symbol
        currency_info = next((info for info in exchange_rates.values() if info['currency_symbol'] == min_currency_symbol), None)

        if currency_info:
            # Convert salaries to USD using exchange rate
            exchange_rate = currency_info['exchange_rate']
            min_salary_usd = round(min_salary / exchange_rate)
            max_salary_usd = round(max_salary / exchange_rate)

            return (min_salary_usd, max_salary_usd)
        else:
            return (None, None)
    else:
        return (None, None)

def extract_job_level(text):
    """
    Extracts job level from a given text.

    Parameters:
    - text (str): The input text.

    Returns:
    - str or None: The extracted job level string. Returns None if no job level is found.
    """
    text = str(text)
    # Define keywords related to job levels
    job_level_keywords = {
        'entry-level': 'entry',
        'junior': 'junior',
        'mid-level': 'mid',
        'senior': 'senior'

    }

    # Create a regular expression pattern to match job level keywords
    pattern = re.compile(r'\b(?:' + '|'.join(re.escape(keyword) for keyword in job_level_keywords.keys()) + r')\b', flags=re.IGNORECASE)

    # Search for job level keywords in the job description
    match = pattern.search(text)

    if match:
        # Return the matched job level string
        return job_level_keywords[match.group().lower()]
    else:
        return None

def process_jobs_df(df):
  """
    Process jobs dataset.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.

    Returns:
    - pd.DataFrame: Processed DataFrame.
  """
  columns = ['site', 'role', 'company_name', 'location', 'job_description',
            'salary_range', 'ratings', 'job_level', 'Search_Keywords',
            'Search_Location', 'Job_Detail_Link', 'Post_Time', 'Current_Time',
            'job_type']
  # drop duolicates
  df.drop_duplicates(inplace = True)

  # remove non-english rows (based on job_description)
  eng_lang = df['job_description'].apply(is_eng)
  df = df[eng_lang]

  # remove senior roles
  df = df[~(df['role'].str.lower().str.contains('sr') | df['role'].str.lower().str.contains('senior'))]

  # process 'site' column: convert to lowercase
  processed_site = df['site'].apply(lambda x: text_preprocessing(x, 'standardized_text'))

  # process 'role', 'company_name', 'location', 'job_description', columns with the following stes:
  # 1)convert to string, 2)remove newline chars, 3) make lowercase, 4) remove punctuation, 5) remove numbers 6) remove stopwords
  processed_role = parallel_processing(df['role'], text_preprocessing, ('no_stopwords'))
  processed_company_name = parallel_processing(df['company_name'], text_preprocessing, 'no_stopwords')
  processed_location = parallel_processing(df['location'], text_preprocessing, 'no_stopwords')
  processed_job_description = parallel_processing(df['job_description'], text_preprocessing, 'no_stopwords')

  # process 'salary_range' column: extract salary (range or number), convert the number into float, convert to USD using the exchange_rates dictionary,
  # add minimum and maximum values in two different columns
  processed_min_salary_usd = list(map(lambda x: x[0], parallel_processing(df['salary_range'], extract_salary)))
  processed_max_salary_usd = list(map(lambda x: x[1], parallel_processing(df['salary_range'], extract_salary)))

  # process 'ratings' column: remove start symbol
  processed_ratings = parallel_processing(df['ratings'], lambda x: str(x).replace(' ★', ''))

  # process 'job_level' column: get job_level in lowercase from job_level column, if it's get job description from role column, otherwise from job_description
  # by targeting the following keywords: 'entry', 'junior', 'mid_level', 'senior'
  processed_job_level = df.apply(lambda row: row['job_level'] if pd.notnull(row['job_level'])
                                                              else extract_job_level(row['role']) if pd.notnull(extract_job_level(row['role']))
                                                              else extract_job_level(row['job_description']), axis=1)

  # process 'Search_Keywords', 'Search_Location' and 'job_type' columns: make lowercase & remove punctuation
  processed_Search_Keywords = parallel_processing(df['Search_Keywords'], text_preprocessing, ('no_punctuation'))
  processed_Search_Location = parallel_processing(df['Search_Location'], text_preprocessing, ('no_punctuation'))
  processed_job_type = parallel_processing(df['job_type'], text_preprocessing, ('no_punctuation'))

  # process 'Job_Detail_Link' function: just convert to string
  processed_Job_Detail_Link = parallel_processing(df['Job_Detail_Link'], text_preprocessing, ('text'))

  # process 'Post_Time', 'Current_Time' columns: make lowercase
  processed_Post_Time = parallel_processing(df['Post_Time'], text_preprocessing, ('standardized_text'))
  processed_Current_Time = parallel_processing(df['Current_Time'], text_preprocessing, ('standardized_text'))

  processed_df = pd.DataFrame({
        'site': processed_site,
        'role': processed_role,
        'company_name': processed_company_name,
        'location': processed_location,
        'job_description': processed_job_description,
        'min_salary': processed_min_salary_usd,
        'max_salary': processed_max_salary_usd,
        'ratings': processed_ratings,
        'job_level': processed_job_level,
        'Search_Keywords': processed_Search_Keywords,
        'Search_Location': processed_Search_Location,
        'Job_Detail_Link': processed_Job_Detail_Link,
        'Post_Time': processed_Post_Time,
        'Current_Time': processed_Current_Time,
        'job_type': processed_job_type
    })
  return processed_df

In [12]:
processed_data = process_jobs_df(data)
processed_data.head()

,site,role,company_name,location,job_description,min_salary,max_salary,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
1,glassdoor,data analyst data entry,juristat,remote,juristat building innovative suite products ne...,30000.0,35000.0,nan,entry,nan,nan,nan,nan,nan,nan
2,glassdoor,business systems analyst,distributed solar development,new york ny,join azad technology partners business systems...,130000.0,160000.0,nan,None,nan,nan,nan,nan,nan,nan
3,glassdoor,data analyst,haberfeld,lincoln ne,haberfeld align marketing products people sust...,54000.0,82000.0,nan,None,nan,nan,nan,nan,nan,nan
6,glassdoor,data analysis charlotte corporate location,genpak llc,charlotte nc,data analyst collect customer requirements det...,52000.0,81000.0,nan,None,nan,nan,nan,nan,nan,nan
7,glassdoor,data science graduate,hewlett packard enterprise,spring tx,data science graduate role designated edge mea...,57000.0,131000.0,nan,None,nan,nan,nan,nan,nan,nan


In [13]:
processed_data.to_csv('preprocessed_combined_dataset.csv')